In [ ]:
from glob import glob
from PipeLine import *
import pandas as pd
import numpy as np

from functools import reduce

import statsmodels.tsa.stattools as tsas
import statsmodels.graphics.tsaplots as tsap

In [ ]:
%%time
# distributions:
database = '/Users/amirhsi_mini/analysis/'
dist_db= database+"/N*-all-analysis-ens_avg/N*.csv"
properties_db  = database+'all_in_one-properties-ens_avg-normalized.csv'
simulation_type = 'all'
geometry='cylindrical'
radial_dists = PipeLine.generator_dist_all_in_one(dist_db, properties_db, simulation_type, geometry, 'radial')
longitudinal_dists = PipeLine.generator_dist_all_in_one(dist_db, properties_db, simulation_type, geometry, 'longitudinal')

In [ ]:
%%time
from dask.distributed import Client
from dask import delayed
from dask import compute
client = Client(n_workers=2, threads_per_worker=1, silence_logs=1)
client
database = '/Users/amirhsi_mini/analysis/'
prop_path= database+"/N*-bug-analysis-ens_avg"
properties_db  = database+'all_in_one-properties-ens_avg-normalized.csv'
simulation_type = 'bug'
geometry='cylindrical'
analysis_delayed = []
attrs_dict = {'rFloryHists': 'r', 'rHists': 'r', 'zHists': 'z','thetaHists': 'theta','rPhis' : 'r',
              'zPhis': 'z','rRhos': 'r', 'zRhos': 'z'}
for attr, direction in attrs_dict.items():
    analysis = delayed(PipeLine.all_in_one_properties)(prop_path, attr, properties_db, simulation_type, geometry, direction = direction)
    analysis_delayed.append(analysis)
results = compute(analysis_delayed)

In [ ]:
%%time
## for a a loop with one core, it takes ~ 4 min
## This does NOT work with more than 2 workers.
database = '/Users/amirhsi_mini/analysis/'
prop_path= database+"/N*-bug-analysis-ens_avg"
properties_db  = database+'all_in_one-properties-ens_avg-normalized.csv'
simulation_type = 'bug'
geometry='cylindrical'
#attrs_dict = {'fsd_t': None, 'gyr_t': None, 'rFlory_t': None}
attrs_dict = {'fsd_t-acf_ci_lower': None, 'gyr_t-acf_ci_lower': None, 'rFlory_t-acf_ci_lower': None, 'fsd_t-acf_ci_upper': None, 'gyr_t-acf_ci_upper': None, 'rFlory_t-acf_ci_upper': None, 'fsd_t-acf_only': None, 'gyr_t-acf_only': None, 'rFlory_t-acf_only': None}
for attr, direction in attrs_dict.items():
    _ = PipeLine.all_in_one_properties(prop_path, attr, properties_db, simulation_type, geometry, direction = None)

In [ ]:
merging_cols = ['time', 'nmon', 'dcyl', 'dcrowd', 'phi_c_bulk', 'phi_c_bulk_eff', 'ens_name', 'group_name']
attrs_names = attrs_dict.keys() # change this to use.
database = '/Users/amirhsi_mini/analysis/'
file_type = 'all_in_one'
simulation_type = 'bug'
db_type = 'ens_avg.csv'
dfs_to_be_merged = []
for attr in attrs_names:
    df_name = database + "-".join([file_type, simulation_type, attr, db_type])
    df = pd.read_csv(df_name, index_col=0)
    dfs_to_be_merged.append(df)
df_merged = reduce(lambda  left, right: pd.merge(left, right, on=merging_cols), dfs_to_be_merged)
df_merged.to_csv(database+"-".join([file_type, simulation_type, "acf_t", db_type]))

In [ ]:
merge_dfs(attrs_names.keys, merging_cols)